# Case 1: MSE
Is this differentiable or not?

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, losses

# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        layers.Input(shape=(5,)),
        layers.Dense(3, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    return model

# Create a simple model
model = create_model()

# Generate some random binary classification data
x = tf.random.uniform((10, 5))
y = tf.random.uniform((10,), minval=0, maxval=2, dtype=tf.int32)

# Calculate MAE loss
def loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    return tf.reduce_sum(tf.abs(y_true - y_pred))

# Now let's try to take derivative of loss
with tf.GradientTape() as tape:
    tape.watch(model.trainable_variables)
    y_pred = model(x, training=True)
    y_pred = tf.round(y_pred)
    acc = loss(y, y_pred) # accuracy(y, y_pred)

grads = tape.gradient(acc, model.trainable_variables)
grads

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

[None, None, None, None]

Ans: Non-Differentiable as `rounding` or `thresholding` is not differentiable due to **discountinuity**

# Case 2: Accuracy

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, losses

# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        layers.Input(shape=(5,)),
        layers.Dense(3, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    return model

# Create a simple model
model = create_model()

# Generate some random binary classification data
x = tf.random.uniform((10, 5))
y = tf.random.uniform((10,), minval=0, maxval=2, dtype=tf.int32)

# Define a simple accuracy calculation function
def accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    accuracy = tf.reduce_mean(y_true*y_pred) + tf.reduce_mean(1. - y_true*y_pred)
    return accuracy

# Now let's try to take derivative of accuracy
with tf.GradientTape() as tape:
    tape.watch(model.trainable_variables)
    y_pred = model(x, training=True)
#     y_pred = tf.round(y_pred)
    acc = accuracy(y, y_pred) # accuracy(y, y_pred)

# This will produce `None` gradient as `rounding` or `thresholding` is not differentiable
grads = tape.gradient(acc, model.trainable_variables)
grads


[<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
 array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>,
 <tf.Tensor: shape=(3, 1), dtype=float32, numpy=
 array([[0.],
        [0.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>]

Ans: Differentiable as no `rounding` or `thresholding`.

# Is it Even Accuracy ? :/

In [3]:
import tensorflow as tf

# Create some binary data
y_true = tf.constant([0, 1, 1, 0, 1, 0], dtype=tf.float32)
y_pred = tf.constant([0.1, 0.9, 0.7, 0.3, 0.8, 0.2], dtype=tf.float32)

# Convert the probabilities into binary predictions
# y_pred = tf.round(y_pred)

# Define a simple accuracy calculation function
def accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    accuracy = tf.reduce_mean(y_true*y_pred) + tf.reduce_mean((1. - y_true)*(1. -y_pred))
    return accuracy

# Calculate accuracy
acc = accuracy(y_true, y_pred)

print("Accuracy:", acc.numpy())


Accuracy: 0.79999995
